In [19]:
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [42]:
market_data = pd.read_csv('소상공인시장진흥공단_상가(상권)정보_서울_202012.csv',sep='|')
market_data.head(2)

,상가업소번호,상호명,지점명,상권업종대분류코드,상권업종대분류명,상권업종중분류코드,상권업종중분류명,상권업종소분류코드,상권업종소분류명,표준산업분류코드,...,건물관리번호,건물명,도로명주소,구우편번호,신우편번호,동정보,층정보,호정보,경도,위도
0,17163092,도전최강달인왕만두,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1174010200102170000018014,고덕그라시움,서울특별시 강동구 고덕로 333,134082,5224.0,NaN,NaN,NaN,127.159471,37.556197
1,17120456,이때,NaN,Q,음식,Q01,한식,Q01A01,한식/백반/한정식,I56111,...,1144012400103900067027687,NaN,서울특별시 마포구 동교로38안길 7,121867,3982.0,NaN,NaN,NaN,126.924660,37.562176


In [21]:
cafe_data = market_data[market_data["상권업종중분류명"] == '커피점/카페']

In [34]:
names = pd.read_csv('D:/02.users/wodud2468/toBigs15_crwaler-master/seoul names.csv', index_col=0)
names['0'] = names['0'] +'카페'
names.head(2)

,0
Unnamed: 0,
0,둔촌2동카페
1,둔촌1동카페


In [35]:
# ChromeDriver 89.0.4389.82(공식 빌드) (64비트)

In [36]:
Chrome_directory = 'D:/02.users/wodud2468/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(Chrome_directory)
options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 

In [38]:
'https://www.diningcode.com/list.php?query='+ names['0'][1]

'https://www.diningcode.com/list.php?query=둔촌1동카페'

In [40]:
url = 'https://www.diningcode.com/list.php?query='+ names['0'][1]
driver.get(url)
# 페이지 소스 가져오기
html=driver.page_source
soup = BeautifulSoup(html)
time.sleep(2)

In [183]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os
from pathlib import Path
import selenium
from selenium import webdriver

Chrome_directory = 'D:/02.users/wodud2468/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(Chrome_directory)
options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 
names = pd.read_csv('D:/02.users/wodud2468/toBigs15_crwaler-master/seoul names.csv', index_col=0) # 서울시 행정구역 정보 불러오기

base_url = "https://www.diningcode.com/2018/ajax/list.php"
total_cafe = []

   
# ---------카페 이름, 링크 크롤링 ----------------
# 각 카페의 이름을 가져옵니다.
for index, data in names[:10].iterrows():
    page = 1 # 한 페이지에 10개씩 보여줍니다.
    while(True):
        url = base_url + '?' + 'query=' + names.iloc[0].item() + '카페' + '&' + 'page=' + str(page) # 주소 설정
        driver.get(url) # 크롬으로 페이지 접속
        html = driver.page_source # 페이지 소스 저장
        soup = BeautifulSoup(html, 'html.parser') # 스프로 파싱
         
        # 데이터에서 텍스트를 추출합니다
        cafe_list = [data.text.split(' ', 1)[1] for data in soup.find_all("span", class_="btxt")]
        cafe_link = [data.get("href") for data in soup.find_all("a", class_='blink')]
        review_counts = [int(data.text) for data in soup.find_all("span", class_="review button")]
            
        # 추출한 데이터를 묶어서 반복합니다.
        for pair in zip(cafe_list, cafe_link, review_counts):
                
            if pair[2] != 0: # 리뷰가 있는 카페만 크롤링합니다.
               # 데이터 저장용 딕셔너리를 생성합니다
                cafe_dict = {
                        'name': pair[0],
                        'link': pair[1]
                }

                    # 만든 딕셔너리가 배열에 없으면 추가합니다.
                if cafe_dict not in total_cafe:
                    total_cafe.append(cafe_dict)
            
        if len(cafe_list) != 10: break # 목록이 10개 미만이면 최대 개수에 도달했다는 뜻입니다.   
        page += 1
            

# 카페 상세정보 link는 uri가 포함되어있습니다.
base_url = "https://www.diningcode.com/"
src = ""

name_list = [] 
main_tag_list = []
score_list = []
star_score = []
like = []
locate = []
taste_score = []
price_score = []
service_score = []
feat_text=[]
purpose_list = []
#---------------------------------
cafe_name = []
user = []
review = []

# 위에서 생성된 total_cafe 배열을 사용해 접근합니다.
# 예외처리 위한 라이브러리
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException

for cafe in total_cafe[:3]:

    # url 생성 후 접근
    print(cafe,':crawling~~~')
    url = base_url + cafe['link']
    driver.get(url)
    #방문목적 더보기 클릭   
    try:
        btn_more = driver.find_element_by_xpath('//*[@id="div_profile"]/div[8]/div[1]/ul[1]/li[1]/span')
        btn_more.click()
        # time.sleep(1)
    except:
        break

    try:
        # 카페명
        name_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[2]/p").text)
    except NoSuchElementException:
        name_list.append('-')
    try:
        #메인태그
        main_tag_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[3]/span").text)    
    except NoSuchElementException:
        main_tag_list.append('-')
    try:
        # 스코어
        score_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[4]/p/strong").text)
    except NoSuchElementException:
        score_list.append('-')
    try:
        # 별점 5점 만점
        star_score.append(driver.find_element_by_xpath("//*[@id='lbl_review_point']").text)
    except NoSuchElementException:
        star_score_list.append('-')
        
    try:
        # 좋아요수
        like.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[1]/div[5]/a[1]/span/i").text)
    except NoSuchElementException:
        like_list.append('-')
    try:
        # 주소
        locate.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[1]").text)
    except NoSuchElementException:
        locate_list.append
    try:
        # 맛 스코어
        taste_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[1]/i").text)
    except NoSuchElementException:
        taste_score.append('-')
    try:
        # 가격 스코어
        price_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[2]/i").text)
    except NoSuchElementException:
        price_score.append('-')
    try:
        # 서비스 스코어
        service_score.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/p[2]/span[3]/i").text)
    except NoSuchElementException:
        service_score.append('-')
    try:
        # 특징
        feat_text.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[2]/ul/li[4]").text)
    except NoSuchElementException:
        feat_text.append('-')
    try:
        # 방문목적
        purpose_list.append(driver.find_element_by_xpath("//*[@id='div_profile']/div[8]/div[1]/ul[1]/li[1]").text)
    except NoSuchElementException:
        purpose_list.append('-')    
    
    #유저 리뷰 크롤링
    #리뷰 더보기 버튼 누르기
    while True:   
        try:
            btn_more = driver.find_element_by_xpath('//*[@id="div_more_review"]')
            btn_more.click()
            # time.sleep(1)
        except:
            break
            
    #리뷰 개수만큼 크롤링하기 위해서 리뷰 개수에 대한 정보가 필요하다
    rev = driver.find_element_by_xpath('//*[@id="div_profile"]/div[8]/p').text
    for i in range(int(rev.split('건')[0])):
        #카페명
        cafe_name.append(cafe['name'])
        #유저ID 및 영향력
        user.append(driver.find_elements_by_xpath("//p[@class='person-grade']")[i].text)
        #리뷰
        review.append(driver.find_elements_by_css_selector('p.review_contents.btxt')[i].text)
        
        
        
print('crawling_done')

{'name': '엘리펀트커피', 'link': '/profile.php?rid=kZDSMy8d7k1O'} :crawling~~~
{'name': '드 까르멜릿', 'link': '/profile.php?rid=pfEZzZ1KBNju'} :crawling~~~
crawling_done


In [184]:
crawling_cafe = pd.DataFrame([name_list,main_tag_list,score_list,star_score,like,locate,taste_score,price_score,service_score,feat_text,purpose_list]).T
crawling_cafe.columns = ['name','main_tag','score','star_score','like','locate','taste_score','price_score','service_score','feat_text','purpose_list']
crawling_cafe

,name,main_tag,score,star_score,like,locate,taste_score,price_score,service_score,feat_text,purpose_list
0,엘리펀트커피,혼카페,33점,4.5점,11,서울특별시 강동구 둔촌2동 53-3,5.0,2.9,5.0,"빈티지, 무료주차",방문목적\n혼카페(2) 차모임(1) 데이트(1) 간식(1) 기념일(1)\n…더보기


In [185]:
user_data = pd.DataFrame([cafe_name,user,review]).T
user_data.columns=['cafe_name','user','review']
user_data

,cafe_name,user,review
0,엘리펀트커피,"maibi (37곳 작성, 84개 공감받음)\n2020년 7월 29일",공부나 업무 보기 좋도록 되어 있습니다. 혼자앉는 자리도 많고 인테리어도 좋습니다.
1,엘리펀트커피,"절대미각장금이 (242곳 작성, 2,300개 공감받음)\n제 입에선 고기를 씹을때 ...",분위기 좋은 카페! 넓고 꽤나 쾌적하고 조용한 편이지만 가격은 좀 높음 ㅠ


In [165]:
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import os
from pathlib import Path
import selenium
from selenium import webdriver

Chrome_directory = 'D:/02.users/wodud2468/chromedriver/chromedriver.exe'
driver = webdriver.Chrome(Chrome_directory)
options = webdriver.ChromeOptions()
options.add_argument('headless')    # 웹 브라우저를 띄우지 않는 headless chrome 옵션 

from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
driver.get('https://www.diningcode.com/profile.php?rid=tQjzQKd3R2jR')
while True:   
    try:
        btn_more = driver.find_element_by_xpath('//*[@id="div_more_review"]')
        btn_more.click()
        # time.sleep(1)
    except:
        break

In [169]:
driver.find_elements_by_xpath("//p[@class='person-grade']")[40]

<selenium.webdriver.remote.webelement.WebElement (session="7c4878c590fb3b1101585ab4cee29d70", element="843debae-cb59-4b59-ac43-90c4f9c1c84b")>

In [172]:
driver.find_elements_by_css_selector('p.review_contents.btxt')[40]

<selenium.webdriver.remote.webelement.WebElement (session="7c4878c590fb3b1101585ab4cee29d70", element="14ac8391-ff16-4ac0-aa35-cbb895312667")>

In [130]:
#리뷰 개수만큼 크롤링하기 위해서 리뷰 개수에 대한 정보가 필요하다
rev = driver.find_element_by_xpath('//*[@id="div_profile"]/div[8]/p').text
for i in range(int(rev.split('건')[0])-1):
    #유저ID 및 영향력
    print(driver.find_elements_by_xpath("//p[@class='person-grade']")[i].text)
    #리뷰
    print(driver.find_elements_by_xpath("//p[@class='review_contents.btxt']")[i].text)

조이 (571곳 작성, 2,195개 공감받음)
함께 즐겨요, 맛있는 음식과 다양한 식감.
2019년 7월 16일


IndexError: list index out of range